<a href="https://colab.research.google.com/github/tsaniarisqi/Machine_Learning_2022/blob/main/UTS/js08_uts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*   Nama  :  Tsania Risqi El Istiqomah
*   Kelas : TI-4J
*   NIM   : 2241727041

## Deteksi Emosi Pengguna Twitter

Deteksi emosi merupakan salah satu permasalahan yang dihadapi pada ***Natural Language Processing*** (NLP). Alasanya diantaranya adalah kurangnya dataset berlabel untuk mengklasifikasikan emosi berdasarkan data twitter. Selain itu, sifat dari data twitter yang dapat memiliki banyak label emosi (***multi-class***). Manusia memiliki berbagai emosi dan sulit untuk mengumpulkan data yang cukup untuk setiap emosi. Oleh karena itu, masalah ketidakseimbangan kelas akan muncul (***class imbalance***). Pada Ujian Tengah Semester (UTS) kali ini, Anda telah disediakan dataset teks twitter yang sudah memiliki label untuk beberapa kelas emosi. Tugas utama Anda adalah membuat model yang mumpuni untuk kebutuhan klasifikasi emosi berdasarkan teks.

### Informasi Data

Dataset yang akan digunakan adalah ****tweet_emotion.csv***. Berikut merupakan informasi tentang dataset yang dapat membantu Anda.

- Total data: 40000 data
- Label emosi: anger, boredom, empty, enthusiasm, fun, happiness, hate, love, neutral, relief, sadness, surprise, worry
- Jumlah data untuk setiap label tidak sama (***class imbalance***)
- Terdapat 3 kolom = 'tweet_id', 'sentiment', 'content'

### Penilaian UTS

UTS akan dinilai berdasaarkan 4 proses yang akan Anda lakukan, yaitu pra pengolahan data, ektraksi fitur, pembuatan model machine learning, dan evaluasi.

#### Pra Pengolahan Data

> **Perhatian**
> 
> Sebelum Anda melakukan sesuatu terhadap data Anda, pastikan data yang Anda miliki sudah "baik", bebas dari data yang hilang, menggunakan tipe data yang sesuai, dan sebagainya.
>

Data tweeter yang ada dapatkan merupakan sebuah data mentah, maka beberapa hal dapat Anda lakukan (namun tidak terbatas pada) yaitu,

1. Case Folding
2. Tokenizing
3. Filtering
4. Stemming

*CATATAN: PADA DATA TWITTER TERDAPAT *MENTION* (@something) YANG ANDA HARUS TANGANI SEBELUM MASUK KE TAHAP EKSTRAKSI FITUR*

#### Ekstrasi Fitur

Anda dapat menggunakan beberapa metode, diantaranya

1. Bag of Words (Count / TF-IDF)
2. N-gram
3. dan sebagainya

#### Pembuatan Model

Anda dibebaskan dalam memilih algoritma klasifikasi. Anda dapat menggunakan algoritma yang telah diajarkan didalam kelas atau yang lain, namun dengan catatan. Berdasarkan asas akuntabilitas pada pengembangan model machine learning, Anda harus dapat menjelaskan bagaimana model Anda dapat menghasilkan nilai tertentu.

#### Evaluasi

Pada proses evaluasi, minimal Anda harus menggunakan metric akurasi. Akan tetapi Anda juga dapat menambahkan metric lain seperti Recall, Precision, F1-Score, detail Confussion Metric, ataupun Area Under Curve (AUC).

### Lembar Pengerjaan
Lembar pengerjaan dimulai dari cell dibawah ini

In [1]:
from google.colab import drive

# Accessing My Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd

In [51]:
df = pd.read_csv('/content/drive/MyDrive/dataset/uts/tweet_emotions.csv')

df.head()

,tweet_id,sentiment,content
0,1956967341,empty,@tiffanylue i know i was listenin to bad habi...
1,1956967666,sadness,Layin n bed with a headache ughhhh...waitin o...
2,1956967696,sadness,Funeral ceremony...gloomy friday...
3,1956967789,enthusiasm,wants to hang out with friends SOON!
4,1956968416,neutral,@dannycastillo We want to trade with someone w...


In [52]:
# cek apakah dalam dataset terdapat nilai yang kosong
df.isnull().sum()

tweet_id     0
sentiment    0
content      0
dtype: int64

#### Pra Pengolahan Data



1.   Drop kolom tweet_id
2.   Remove username
3.   Remove tanda baca
4.   Remove links



##### Drop kolom

In [53]:
df = df.drop(['tweet_id'],axis=1)
df.head()

,sentiment,content
0,empty,@tiffanylue i know i was listenin to bad habi...
1,sadness,Layin n bed with a headache ughhhh...waitin o...
2,sadness,Funeral ceremony...gloomy friday...
3,enthusiasm,wants to hang out with friends SOON!
4,neutral,@dannycastillo We want to trade with someone w...


##### Remove username

In [54]:
import re
import nltk
nltk.download('omw-1.4')
from nltk.corpus import stopwords 
from textblob import Word

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [55]:
STOPWORDS = stopwords.words("english")

In [56]:
def clean(text):
    text = text.lower()
    text = re.sub("[^\w\s]","",text) # Remove punctuations 
    text = " ".join(w for w in text.split() if w not in STOPWORDS)
    text = " ".join([Word(word).lemmatize() for word in text.split()])
    return text

In [63]:
df['content'] = df['content'].apply(lambda x : clean(x)) 

In [64]:
freq=  pd.Series(" ".join(df['content']).split()).value_counts()[-10000:] 

In [65]:
freq =  list(freq.index)

In [66]:
df['content'] = df['content'].apply(lambda x:  " ".join(x for x in x.split() if x not in freq))

In [68]:
df.head()

,sentiment,content
0,empty,tiffanylue know listenin bad habit earlier sta...
1,sadness,layin n bed headache call
2,sadness,funeral friday
3,enthusiasm,want hang friend soon
4,neutral,want trade someone houston ticket one


#### Ekstrasi Fitur

In [79]:
# Bag of Words (TF-IDF)

from sklearn.model_selection import train_test_split 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder

x = np.array(df['content'].values)[:20000]
y =  np.array(df['sentiment'].values)[:20000]

Le = LabelEncoder()
y =  Le.fit_transform(y)

x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,shuffle=True,random_state=42,stratify=y)

tf = TfidfVectorizer(analyzer='word',max_features=1000,ngram_range=(1,3))
x_tf = tf.fit_transform(x_train)
x_val_tf = tf.transform(x_test)

x_tf   = x_tf.toarray()
x_val_tf =  x_val_tf.toarray()

#### Pembuatan Model Machine Learning

In [81]:
# menggunakan algoritma Multinomial Naive Bayes
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

# Inisiasi MultinomialNB
mnb = MultinomialNB()

# Fit model
mnb.fit(x_tf, y_train)

# Prediksi dengan data training
y_pred_train = mnb.predict(x_tf)

# Evaluasi akurasi data training
acc_train = accuracy_score(y_train, y_pred_train)

# Prediksi dengan data training
y_pred_test = mnb.predict(x_val_tf)

# Evaluasi akurasi data training
acc_test = accuracy_score(y_test, y_pred_test)

# Print hasil evaluasi
print(f'Hasil akurasi data train: {acc_train}')
print(f'Hasil akurasi data test: {acc_test}')

Hasil akurasi data train: 0.385625
Hasil akurasi data test: 0.332


#### Evaluasi

In [90]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, recall_score, precision_score, f1_score

In [94]:
print("MultinomialNB")
print('Accuracy  = ', round(accuracy_score(y_test, y_pred_test)*100,2),'%')
print('Recall    = ', round(recall_score(y_test, y_pred_test, average="weighted")*100,2),'%')
print('Precision = ', round(precision_score(y_test, y_pred_test, average="weighted")*100,2),'%')
print('F1-Score  = ', round(f1_score(y_test, y_pred_test, average="weighted")*100,2),'%')

print("")
print("MultinomialNB")
print(classification_report(y_test,y_pred_test))  


MultinomialNB
Accuracy  =  33.2 %
Recall    =  33.2 %
Precision =  32.17 %
F1-Score  =  25.11 %

MultinomialNB
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        16
           1       0.00      0.00      0.00        28
           2       0.00      0.00      0.00        93
           3       0.00      0.00      0.00        52
           4       0.00      0.00      0.00        93
           5       0.00      0.00      0.00       180
           6       0.58      0.03      0.06       212
           7       0.33      0.01      0.01       158
           8       0.24      0.14      0.18       740
           9       0.00      0.00      0.00        87
          10       0.33      0.26      0.29       886
          11       1.00      0.00      0.01       212
          12       0.35      0.79      0.48      1243

    accuracy                           0.33      4000
   macro avg       0.22      0.10      0.08      4000
weighted avg       0.32

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: 

Model Sentiment Analysis diatas mencapai akurasi sekitar 33.2 % untuk prediksi sentimen menggunakan Naive Bayes.